In [1]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
transactions = pd.read_csv("card_data/transactions_data.csv")
print(transactions.shape)
transactions.head()


(13305915, 12)


,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors
0,7475327,2010-01-01 00:01:00,1556,2972,$-77.00,Swipe Transaction,59935,Beulah,ND,58523.0,5499,NaN
1,7475328,2010-01-01 00:02:00,561,4575,$14.57,Swipe Transaction,67570,Bettendorf,IA,52722.0,5311,NaN
2,7475329,2010-01-01 00:02:00,1129,102,$80.00,Swipe Transaction,27092,Vista,CA,92084.0,4829,NaN
3,7475331,2010-01-01 00:05:00,430,2860,$200.00,Swipe Transaction,27092,Crown Point,IN,46307.0,4829,NaN
4,7475332,2010-01-01 00:06:00,848,3915,$46.41,Swipe Transaction,13051,Harwood,MD,20776.0,5813,NaN


In [3]:
cards = pd.read_csv("card_data/cards_data.csv")
print(cards.shape)
cards.head()


(6146, 13)


,id,client_id,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web
0,4524,825,Visa,Debit,4344676511950444,12/2022,623,YES,2,$24295,09/2002,2008,No
1,2731,825,Visa,Debit,4956965974959986,12/2020,393,YES,2,$21968,04/2014,2014,No
2,3701,825,Visa,Debit,4582313478255491,02/2024,719,YES,2,$46414,07/2003,2004,No
3,42,825,Visa,Credit,4879494103069057,08/2024,693,NO,1,$12400,01/2003,2012,No
4,4659,825,Mastercard,Debit (Prepaid),5722874738736011,03/2009,75,YES,1,$28,09/2008,2009,No


In [5]:
users = pd.read_csv("card_data/users_data.csv")
print(users.shape)
users.head()


(2000, 14)


,id,current_age,retirement_age,birth_year,birth_month,gender,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards
0,825,53,66,1966,11,Female,462 Rose Lane,34.15,-117.76,$29278,$59696,$127613,787,5
1,1746,53,68,1966,12,Female,3606 Federal Boulevard,40.76,-73.74,$37891,$77254,$191349,701,5
2,1718,81,67,1938,11,Female,766 Third Drive,34.02,-117.89,$22681,$33483,$196,698,5
3,708,63,63,1957,1,Female,3 Madison Street,40.71,-73.99,$163145,$249925,$202328,722,4
4,1164,43,70,1976,9,Male,9620 Valley Stream Drive,37.76,-122.44,$53797,$109687,$183855,675,1


## Load the MCC codes — industry standard codes describing merchant types (e.g., grocery store, restaurant).

In [7]:
with open("card_data/mcc_codes.json", "r") as f:
    mcc_codes = json.load(f)
# Optionally convert to DataFrame
mcc_df = pd.DataFrame(list(mcc_codes.items()), columns=['mcc_code', 'description'])
print(mcc_df.head())


  mcc_code                           description
0     5812         Eating Places and Restaurants
1     5541                      Service Stations
2     7996  Amusement Parks, Carnivals, Circuses
3     5411          Grocery Stores, Supermarkets
4     4784                 Tolls and Bridge Fees


In [9]:
import json

with open("card_data/train_fraud_labels.json", "r") as f:
    labels_raw = json.load(f)
    
# The actual label mapping is inside the 'target' key!
label_dict = labels_raw['target']
print(type(label_dict))  # Should now be <class 'dict'>
print(list(label_dict.items())[:5])  # Show the first 5 transaction IDs and labels


<class 'dict'>
[('10649266', 'No'), ('23410063', 'No'), ('9316588', 'No'), ('12478022', 'No'), ('9558530', 'No')]


In [11]:
import pandas as pd

# Convert dictionary to DataFrame
fraud_labels = pd.DataFrame(list(label_dict.items()), columns=['transaction_id', 'is_fraud'])

# Convert label "Yes"/"No" to 1/0 for ML tasks
fraud_labels['is_fraud'] = fraud_labels['is_fraud'].map({'Yes': 1, 'No': 0})

# Ensure transaction_id is string type for safe merging
fraud_labels['transaction_id'] = fraud_labels['transaction_id'].astype(str)

print(fraud_labels.tail())


        transaction_id  is_fraud
8914958       14064699         0
8914959        7676538         0
8914960       15131030         0
8914961       17244732         0
8914962       15151926         0


In [14]:
print(transactions.columns)


Index(['id', 'date', 'client_id', 'card_id', 'amount', 'use_chip',
       'merchant_id', 'merchant_city', 'merchant_state', 'zip', 'mcc',
       'errors'],
      dtype='object')


In [15]:
transactions = transactions.rename(columns={'id': 'transaction_id'})


In [16]:
transactions['transaction_id'] = transactions['transaction_id'].astype(str)
fraud_labels['transaction_id'] = fraud_labels['transaction_id'].astype(str)


In [19]:
transactions = transactions.merge(fraud_labels, on='transaction_id', how='left')
transactions['is_fraud'].fillna(0, inplace=True)
transactions['is_fraud'] = transactions['is_fraud'].astype(int)


/tmp/ipykernel_144904/3780541298.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  transactions['is_fraud'].fillna(0, inplace=True)


In [20]:
print(transactions['is_fraud'].value_counts())
print("Fraud rate:", transactions['is_fraud'].mean())


is_fraud
0    13292583
1       13332
Name: count, dtype: int64
Fraud rate: 0.001001960406330568


In [ ]:
import pandas as pd
import numpy as np
import json

# === STEP 1: Load All Data ===

transactions = pd.read_csv("card_data/transactions_data.csv")
cards = pd.read_csv("card_data/cards_data.csv")
users = pd.read_csv("card_data/users_data.csv")

# Rename 'id' in cards to 'card_id' for the merge to work
if 'id' in cards.columns:
    cards.rename(columns={'id': 'card_id'}, inplace=True)

if 'id' in users.columns:
    users.rename(columns={'id': 'client_id'}, inplace=True)

# MCC codes
with open("card_data/mcc_codes.json", "r") as f:
    mcc_codes = json.load(f)
mcc_df = pd.DataFrame(list(mcc_codes.items()), columns=['mcc_code', 'description'])

# Fraud labels
with open("card_data/train_fraud_labels.json", "r") as f:
    labels_raw = json.load(f)
label_dict = labels_raw['target']  # inside the top-level 'target' key

fraud_labels = pd.DataFrame(list(label_dict.items()), columns=['transaction_id', 'is_fraud'])
fraud_labels['is_fraud'] = fraud_labels['is_fraud'].map({'Yes': 1, 'No': 0})
fraud_labels['transaction_id'] = fraud_labels['transaction_id'].astype(str)

# Rename 'id' in transactions, if needed
if 'id' in transactions.columns:
    transactions.rename(columns={'id': 'transaction_id'}, inplace=True)
transactions['transaction_id'] = transactions['transaction_id'].astype(str)

# === STEP 2: Merge All Data ===

# 1. Merge fraud labels
transactions = transactions.merge(fraud_labels, on='transaction_id', how='left')
transactions['is_fraud'] = transactions['is_fraud'].fillna(0).astype(int)

# 2. Merge cards info
transactions = transactions.merge(cards, on='card_id', how='left')

# 3. Merge users info
transactions = transactions.merge(users, on='client_id', how='left')

# 4. Merge MCC description
transactions = transactions.merge(mcc_df, left_on='mcc', right_on='mcc_code', how='left')
transactions.drop('mcc_code', axis=1, inplace=True)

# === STEP 3: Feature Engineering ===

# Dates
transactions['date'] = pd.to_datetime(transactions['date'])
transactions['hour'] = transactions['date'].dt.hour
transactions['dayofweek'] = transactions['date'].dt.dayofweek

# Amount (log scale)
transactions['log_amount'] = np.log1p(transactions['amount'])

# Account open date (years since open)
if 'acct_open_date' in transactions.columns:
    transactions['acct_open_date'] = pd.to_datetime(transactions['acct_open_date'], errors='coerce')
    transactions['acct_age_yrs'] = (transactions['date'] - transactions['acct_open_date']).dt.days / 365
else:
    transactions['acct_age_yrs'] = np.nan

# Encode categorical/yes-no variables
if 'use_chip' in transactions.columns:
    transactions['use_chip'] = (transactions['use_chip'].astype(str).str.lower().str.strip() == 'yes').astype(int)
if 'has_chip' in transactions.columns:
    transactions['has_chip'] = (transactions['has_chip'].astype(str).str.lower().str.strip() == 'yes').astype(int)
if 'card_on_dark_web' in transactions.columns:
    transactions['card_on_dark_web'] = (transactions['card_on_dark_web'].astype(str).str.lower().str.strip() == 'yes').astype(int)
if 'gender' in transactions.columns:
    transactions['gender'] = (transactions['gender'].astype(str).str.lower().str.strip() == 'male').astype(int)

# One-hot encode categorical variables (card_type, merchant description)
cat_cols = []
if 'card_type' in transactions.columns:
    cat_cols.append('card_type')
if 'description' in transactions.columns:
    cat_cols.append('description')
if cat_cols:
    transactions = pd.get_dummies(transactions, columns=cat_cols, drop_first=True)

# Drop unmappable/free text, ID, and direct date columns for modeling
drop_cols = [
    'transaction_id', 'client_id', 'card_id', 'merchant_id', 'merchant_city', 'merchant_state',
    'zip', 'date', 'acct_open_date', 'errors'
]
transactions.drop([col for col in drop_cols if col in transactions.columns], axis=1, inplace=True)

# === STEP 4: Prepare Features and Target ===

X = transactions.drop('is_fraud', axis=1)
y = transactions['is_fraud']

# --- Split for modeling ---
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# --- Handle class imbalance with SMOTE ---
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=42)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

print("Final training set shape:", X_train_res.shape)
print("Final test set shape:", X_test.shape)
print("Balanced training class counts:\n", pd.Series(y_train_res).value_counts())

# === Optional: Ready for any scikit-learn compatible ML classifier ===


In [24]:
print("transactions columns:", transactions.columns)
print("cards columns:", cards.columns)


transactions columns: Index(['transaction_id', 'date', 'client_id', 'card_id', 'amount', 'use_chip',
       'merchant_id', 'merchant_city', 'merchant_state', 'zip', 'mcc',
       'errors', 'is_fraud'],
      dtype='object')
cards columns: Index(['id', 'client_id', 'card_brand', 'card_type', 'card_number', 'expires',
       'cvv', 'has_chip', 'num_cards_issued', 'credit_limit', 'acct_open_date',
       'year_pin_last_changed', 'card_on_dark_web'],
      dtype='object')
